### 이미지 포함 문서 요약 및 질의응답 처리 에이전트

In [1]:
import os
from dotenv import load_dotenv

# .env 파일의 내용 불러오기
load_dotenv("C:/env/.env")

True

In [2]:
"""
이미지 포함 문서 요약 및 질의응답 에이전트 (LangChain v1.0 + OpenAI API)

폴더 구조 예시
.
├─ vision_rag_agent.py
└─ docs/
   ├─ sample.pdf
   ├─ figure1.png
   └─ scan_page.jpg
"""

import os
import io
import base64
from typing import List, Tuple
from dataclasses import dataclass

import fitz  # PyMuPDF
from PIL import Image

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# --------- 환경 설정 ----------
OPENAI_VISION_MODEL = "gpt-4o-mini"
OPENAI_TEXT_MODEL = "gpt-4o-mini"
EMBED_MODEL = "text-embedding-3-small"
CHROMA_DIR = "chroma_store"

# --------- 공통 LLM/임베딩 ----------
llm = ChatOpenAI(model=OPENAI_TEXT_MODEL, temperature=0.1)
emb = OpenAIEmbeddings(model=EMBED_MODEL)

# --------- 유틸: 이미지 바이트 -> PNG base64 ----------
def to_png_bytes(img: Image.Image) -> bytes:
    buf = io.BytesIO()
    img.save(buf, format="PNG")
    return buf.getvalue()

def pil_from_bytes(raw: bytes) -> Image.Image:
    return Image.open(io.BytesIO(raw)).convert("RGB")

def b64_png(raw: bytes) -> str:
    return base64.b64encode(raw).decode("utf-8")

# --------- OpenAI Vision 캡셔닝 ----------
def caption_image_bytes(raw_bytes: bytes, prompt: str = "이미지의 핵심 내용을 간결히 설명하라. 글머리표 없이 1~3문장으로 요약하라.") -> str:
    """
    ChatOpenAI 래퍼는 텍스트 중심 설계이므로, 이미지 캡셔닝은 messages에 image_url(data URI)을 직접 넣어 호출한다.
    """
    data_uri = f"data:image/png;base64,{b64_png(raw_bytes)}"
    # LangChain 래퍼의 .invoke에 멀티모달 content를 직접 전달
    resp = llm.invoke(
        [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": data_uri}},
                ],
            }
        ]
    )
    return resp.content.strip()

# --------- 문서 로드: PDF에서 텍스트와 이미지 추출 ----------
@dataclass
class PageExtraction:
    page_number: int
    text: str
    image_captions: List[str]

def extract_from_pdf(pdf_path: str) -> List[PageExtraction]:
    doc = fitz.open(pdf_path)
    results: List[PageExtraction] = []
    for i, page in enumerate(doc):
        text = page.get_text("text") or ""
        captions = []
        # 페이지 내 이미지 객체 순회
        for img_index, img in enumerate(page.get_images(full=True) or []):
            xref = img[0]
            pix = fitz.Pixmap(doc, xref)
            if pix.alpha:  # RGBA -> RGB
                pix = fitz.Pixmap(fitz.csRGB, pix)
            raw = pix.tobytes("png")
            try:
                cap = caption_image_bytes(raw)
                captions.append(f"[이미지{i+1}-{img_index+1}] {cap}")
            except Exception as e:
                captions.append(f"[이미지{i+1}-{img_index+1}] 캡션 실패: {e}")
        results.append(PageExtraction(page_number=i + 1, text=text, image_captions=captions))
    doc.close()
    return results

# --------- 단일 이미지 파일 로드 ----------
def extract_from_image(path: str) -> List[PageExtraction]:
    with open(path, "rb") as f:
        raw = f.read()
    cap = caption_image_bytes(raw)
    # 이미지 단독 문서도 page처럼 포맷을 맞춰 반환
    return [PageExtraction(page_number=1, text="", image_captions=[f"[이미지] {cap}"])]

# --------- Document 생성 ----------
def build_documents(paths: List[str]) -> List[Document]:
    docs: List[Document] = []
    for p in paths:
        name = os.path.basename(p)
        ext = os.path.splitext(p.lower())[1]
        if ext in [".pdf"]:
            pages = extract_from_pdf(p)
            for pe in pages:
                combined = ""
                if pe.text.strip():
                    combined += f"[본문 p.{pe.page_number}]\n{pe.text.strip()}\n\n"
                if pe.image_captions:
                    combined += "[이미지 캡션]\n" + "\n".join(pe.image_captions) + "\n"
                if combined.strip():
                    docs.append(
                        Document(
                            page_content=combined.strip(),
                            metadata={"source": name, "page": pe.page_number},
                        )
                    )
        elif ext in [".png", ".jpg", ".jpeg", ".webp", ".bmp"]:
            pages = extract_from_image(p)
            for pe in pages:
                combined = ""
                if pe.image_captions:
                    combined += "[이미지 캡션]\n" + "\n".join(pe.image_captions) + "\n"
                docs.append(
                    Document(
                        page_content=combined.strip(),
                        metadata={"source": name, "page": pe.page_number},
                    )
                )
        else:
            # 단순 텍스트 파일도 허용
            if ext in [".txt", ".md"]:
                with open(p, "r", encoding="utf-8", errors="ignore") as f:
                    content = f.read()
                docs.append(
                    Document(
                        page_content=content.strip(),
                        metadata={"source": name, "page": 1},
                    )
                )
    return docs

# --------- 인덱싱 ----------
def build_vectorstore(docs: List[Document]) -> Chroma:
    splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=150)
    chunks = splitter.split_documents(docs)
    vs = Chroma.from_documents(documents=chunks, embedding=emb, persist_directory=CHROMA_DIR)
    return vs

# --------- 요약 체인 (맵-리듀스 스타일) ----------
def summarize_docs(docs: List[Document]) -> str:
    map_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "너는 전문 문서 요약가이다. 텍스트와 이미지 캡션을 모두 고려해 핵심을 압축하여 메모 형태로 요약하라."),
            ("human", "다음 내용을 5~8개의 항목으로 핵심만 요약하라.\n\n{chunk}"),
        ]
    )
    reduce_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "너는 여러 요약을 하나로 통합하는 편집자이다."),
            ("human", "다음 부분 요약들을 서로 겹치지 않게 통합 요약하라.\n\n{chunks}"),
        ]
    )

    map_chain = map_prompt | llm | StrOutputParser()
    reduce_chain = reduce_prompt | llm | StrOutputParser()

    # 간단한 수동 맵-리듀스
    splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=0)
    texts = [d.page_content for d in docs]
    merged = "\n\n".join(texts)
    parts = splitter.split_text(merged)

    partial_summaries = [map_chain.invoke({"chunk": p}) for p in parts]
    final_summary = reduce_chain.invoke({"chunks": "\n\n".join(partial_summaries)})
    return final_summary.strip()

# --------- 질의응답 체인 (RAG) ----------
def build_rag_chain(vs: Chroma):
    retriever = vs.as_retriever(search_kwargs={"k": 4})

    qa_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                (
                    "너는 문서 기반 어시스턴트이다. 항상 주어진 컨텍스트만을 근거로 대답하라. "
                    "근거가 없으면 모른다고 답하고, 가능한 경우 인용 출처(source, page)를 함께 제시하라."
                ),
            ),
            (
                "human",
                "질문: {question}\n\n다음은 검색된 컨텍스트이다:\n{context}\n\n규칙: 1) 추론을 꾸미지 말라 2) 한국어로 답하라",
            ),
        ]
    )

    def format_docs(docs: List[Document]) -> str:
        out = []
        for d in docs:
            src = d.metadata.get("source")
            page = d.metadata.get("page")
            out.append(f"[{src} p.{page}]\n{d.page_content}")
        return "\n\n".join(out)

    # LCEL: 병렬로 retrieve와 질문 전달
    setup = RunnableParallel(
        context=retriever | (lambda docs: format_docs(docs)),
        question=RunnablePassthrough(),
    )
    rag_chain = setup | qa_prompt | llm | StrOutputParser()
    return rag_chain

# --------- 실행 예시 ----------
def main():
    docs_dir = "docs"
    paths = [os.path.join(docs_dir, f) for f in os.listdir(docs_dir)] if os.path.isdir(docs_dir) else []
    if not paths:
        print("docs 폴더에 PDF나 이미지 파일을 넣은 뒤 다시 실행한다.")
        return

    print("1) 문서 로드 및 이미지 캡셔닝 진행 중...")
    docs = build_documents(paths)
    print(f"   - 생성된 Document 개수: {len(docs)}")

    print("2) 벡터스토어 빌드 및 저장 중...")
    vs = build_vectorstore(docs)
    print(f"   - Chroma 디렉터리: {CHROMA_DIR}")

    # 요약
    print("\n3) 전체 요약 생성 중...")
    summary = summarize_docs(docs)
    print("\n[문서 요약]\n")
    print(summary)

    # 질의응답
    rag = build_rag_chain(vs)
    print("\n4) 질의응답 테스트 예시")
    sample_questions = [
        "문서의 핵심 결론은 무엇인가?",
        "그림이나 도표가 설명하는 핵심 메시지는 무엇인가?",
        "한계나 주의사항이 언급되었는가?",
    ]
    for q in sample_questions:
        print(f"\nQ: {q}")
        ans = rag.invoke(q)
        print(f"A: {ans}")

if __name__ == "__main__":
    main()


1) 문서 로드 및 이미지 캡셔닝 진행 중...
   - 생성된 Document 개수: 9
2) 벡터스토어 빌드 및 저장 중...
   - Chroma 디렉터리: chroma_store

3) 전체 요약 생성 중...

[문서 요약]

**통합 요약**

본 문서는 엣지 컴퓨팅의 필요성과 구조, 관련 기술 및 프로젝트에 대한 포괄적인 개요를 제공한다. 엣지 컴퓨팅은 스마트 기기와 IoT 단말의 증가로 인해 발생하는 데이터 전송 및 처리 지연 문제를 해결하기 위해 등장했으며, 네트워크의 가장자리에서 클라우드 컴퓨팅 기술을 제공하여 데이터 수집 및 분석을 사용자 단말 근처에서 수행함으로써 네트워크 자원을 절약하고 데이터 전송 시간을 단축한다. 

MEC(Mobile Edge Computing) Reference Architecture는 엣지 컴퓨팅을 위한 프레임워크로, Mobile Edge Host Management와 Mobile Edge System Management로 구성된다. MEC 플랫폼은 가상화 인프라에서 MEC 애플리케이션을 실행하고, 서비스 관리를 위한 기능을 제공하며, OPNFV와 같은 오픈소스 프로젝트와의 연계를 통해 엣지 클라우드의 요구사항을 수용하기 위한 작업이 진행되고 있다. 

StarlingX와 Akraino Edge Stack은 엣지 클라우드 환경을 위한 주요 플랫폼으로, 각각 고가용성, QoS, 보안, 저지연성을 제공하며, 다양한 관리 기능과 배포 시나리오를 지원한다. Living Edge Lab은 오픈 엣지 컴퓨팅 플랫폼으로 다양한 테스트 환경을 제공하며, MobiledgeX는 ETSI MEC 표준에 따라 엣지 클라우드 인프라를 구성하고 모바일 단말에 근접한 애플리케이션 배포 구조를 제시한다.

현재 엣지 컴퓨팅 기술은 연구 중이며, 기존 클라우드와 비교해 명확한 스펙이나 플랫폼이 부족하지만, 많은 클라우드 사업자들이 실제 서비스 플랫폼을 제공하고 있으며, 표준 단체 및 오픈소스 플랫폼에서 엣지 클라우드 구조가 설계되고 구현되고 